In [1]:
import os

from langchain.chains import RetrievalQA
from PyPDF2 import PdfReader
from langchain.callbacks.base import BaseCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Neo4jVector
from langchain_community.chat_models import ChatOllama
from langchain_core.documents import Document
import os
import json
from semantic_text_splitter import TextSplitter
# load api key lib
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceHubEmbeddings
embeddings_options=["medllama2","huggingface"]

embeddings_name=embeddings_options[0]

load_dotenv(".env")

url = "neo4j://localhost:7687"
# url = "neo4j://localhost:7474"
username ="neo4j"
password = "password"

os.environ["NEO4J_URI"] = url #"bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] =username # "neo4j"
os.environ["NEO4J_PASSWORD"] = password #"password"


url = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
ollama_base_url = os.getenv("OLLAMA_BASE_URL")
embedding_model_name = os.getenv("EMBEDDING_MODEL")
llm_name = "medllama2"#os.getenv("LLM")
# Remapping for Langchain Neo4j integration
os.environ["NEO4J_URL"] = url



if(embeddings_name==embeddings_options[0]):
    embeddings = OllamaEmbeddings(
                model="medllama2"
            )
if(embeddings_name==embeddings_options[1]):
    # model = "sentence-transformers/all-mpnet-base-v2"
    model = "ncbi/MedCPT-Query-Encoder"
    embeddings = HuggingFaceHubEmbeddings(
    model=model,
    task="embedding",
    huggingfacehub_api_token="xxx",
)






max_characters = 100
# Optionally can also have the splitter not trim whitespace for you
splitter = TextSplitter()

# Document(page_content=sect['text'])

folder_path = "/workspaces/thyroid_knowledge_graph/preprocessed"
file_paths = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)

res=[]
for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        sections= data['sections']
        for sect in sections:
            # res.append(sect['text'])
            for chunk in splitter.chunks(sect['text'], max_characters):

                res.append(chunk)

llm_med = ChatOllama(model="medllama2")




In [2]:
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='medllama2', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [3]:
chunks = res[0:2]
chunks


['Thyroid cancer is divided into differentiated thyroid cancer (DTC) and undifferentiated thyroid',
 'cancer.']

In [4]:
# Store the chunks part in db (vector)

vectorstore = Neo4jVector.from_texts(
    chunks,
    url=url,
    username=username,
    password=password,
    embedding=embeddings,
    index_name="pdf_bot",
    node_label="PdfBotChunk",
    pre_delete_collection=True,  # Delete existing PDF data
)


In [5]:
vector_qa = RetrievalQA.from_chain_type(
    llm=llm_med,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)


In [6]:
vector_qa.run(
    "what is RIT"
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
